<a href="https://colab.research.google.com/github/MachineNeyarning/ClassifiersCommittee/blob/main/ClassifiersCommittee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comitê de Classificadores | Projeto N1

**GitHub: https://github.com/MachineNeyarning/ClassifiersCommittee**

---

## Grupo:
- Gustavo Henrique Martins
- Ícaro Botelho
- Maria Clara Seixa Scheffel
- Maruan Biasi El Achkar
- Ricardo Falcão Schlieper

### Dataset escolhido: Rain in Australia
- Detalhamento de colunas em: https://github.com/MachineNeyarning/ClassifiersCommittee/tree/main/dataset
- Fonte: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package
- Target: RainTomorrow

### Algoritmos escolhidos:
- K-Nearest Neighbors (KNN)
- Decision Tree Classifier (Tree-Based)
- Random Forest Classifier (Tree-Based)
- LightGBM Classifier (Tree-Based)
- CatBoost Classifier (Tree-Based)
- Gaussian Naive Bayes (Naive Bayes)
- Support Vector Classification (SVC)
- Multi-Layer Perceptron (MLPClassifier)
- AdaBoost Classifier (Ensemble Methods)
- Quadratic Discriminant Analysis (QDA)



---



In [20]:
# Baixar Dataset

import kagglehub
import os

path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")
csv_path = os.path.join(path, 'weatherAUS.csv')


Using Colab cache for faster access to the 'weather-dataset-rattle-package' dataset.


In [21]:
# Criar Dataframe Pandas

import pandas as pd

df = pd.read_csv(csv_path, sep=',', engine='python', on_bad_lines='skip')
df.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No


In [22]:
# Describe Dataframe Data

df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [23]:
# Verificar falta de dados

missing_data = df.isnull().sum()
display(missing_data)

,0
Date,0
Location,0
MinTemp,1485
MaxTemp,1261
Rainfall,3261
Evaporation,62790
Sunshine,69835
WindGustDir,10326
WindGustSpeed,10263
WindDir9am,10566


In [24]:
# Remover linhas com dados faltando

df_cleaned = df.dropna()
display(df_cleaned.head())
print(f"Original: {df.shape}")
print(f"Limpo: {df_cleaned.shape}")

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,...,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No


Original: (145460, 23)
Limpo: (56420, 23)


---
# 10 Algoritimos:

In [25]:
# K-Nearest Neighbors (KNN) |

# Copia do dataframe
df_knn = df_cleaned

In [26]:
# Decision Tree Classifier (Tree-Based) |

# Copia do dataframe
df_dtc = df_cleaned

In [27]:
# Random Forest Classifier (Tree-Based) |

# Copia do dataframe
df_rfc = df_cleaned

In [35]:
# LightGBM Classifier (Tree-Based) | Maruan

# Copia do dataframe
df_lgbm = df_cleaned

!pip install lightgbm

import lightgbm as lgb
import sklearn, sys

# Modelo nao vai usar data
df_lightgbm = df_cleaned.drop('Date', axis=1)

# Conveter colunas categoricas para numericas
df_processed = pd.get_dummies(df_lightgbm, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow'], drop_first=True)


X = df_processed.drop('RainTomorrow_Yes', axis=1)
y = df_processed['RainTomorrow_Yes']

display(X.head())
display(y.head())


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_Yes
6049,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,...,False,False,False,False,False,True,False,False,False,False
6050,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,...,False,False,False,True,False,False,False,False,False,False
6052,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,...,False,False,False,False,False,False,False,False,False,False
6053,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,...,False,False,False,False,False,False,False,False,True,False
6054,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,...,False,False,False,False,False,False,False,True,False,False


,RainTomorrow_Yes
6049,False
6050,False
6052,False
6053,False
6054,False


In [29]:
# CatBoost Classifier (Tree-Based) | Maruan

# Copia do dataframe
df_cb = df_cleaned

In [30]:
# Gaussian Naive Bayes (Naive Bayes) |

# Copia do dataframe
df_gnb = df_cleaned

In [31]:
# Support Vector Classification (SVC) |

# Copia do dataframe
df_svc = df_cleaned

In [32]:
# Multi-Layer Perceptron (MLPClassifier) |

# Copia do dataframe
df_mlp = df_cleaned

In [33]:
# AdaBoost Classifier (Ensemble Methods) |

# Copia do dataframe
df_ada = df_cleaned

In [34]:
# Quadratic Discriminant Analysis (QDA) |

# Copia do dataframe
df_qda = df_cleaned

---
# Comite Classificador: